In [1]:
import pandas as pd
import altair as alt
import folium
import folium.plugins
import branca.colormap as cm
import folium.features

In [3]:
alt.themes.enable('dark')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
EQ_data = pd.read_csv("ISC_catalog_1950_1999.csv", usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
EQ_data = EQ_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/tmp/ipykernel_33248/1694919827.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  EQ_data = EQ_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [5]:
EQ_data['TYPE'].value_counts()

TYPE
ke    1386539
fe      46999
se      15254
kh      14863
kr       3227
de       1631
sh       1493
kn       1077
km        286
sm        220
kx         88
ki         73
si         19
uk          1
sn          1
sx          1
di          1
sq          1
kq          1
Name: count, dtype: int64

In [6]:
mask = ((EQ_data['TYPE'] == 'kn') & (EQ_data['MAG'].isna() != True))
NU_data = EQ_data[mask]

NU_data.shape

(946, 12)

In [7]:
tile_World_Imagery = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
attr_World_Imagery = 'Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'

tile_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
attr_WorldStreetMap = 'Tiles &copy; Esri &mdash; Source: Esri, DeLorme, NAVTEQ, USGS, Intermap, iPC, NRCAN, Esri Japan, METI, Esri China (Hong Kong), Esri (Thailand), TomTom, 2012'

In [8]:
min_lon, max_lon = 122, 154
min_lat, max_lat = 20, 45

In [9]:
m = folium.Map(
    # max_bounds=True,
    tiles=tile_WorldStreetMap, 
    attr=attr_WorldStreetMap,
    zoom_control=False,
    control_scale=True, 
    zoom_start=1,
    # min_lat=20,
    # max_lat=45,
    # min_lon=122,
    # max_lon=154,
    
    )

In [10]:
colormap = cm.linear.inferno.scale(2,7)
colormap.caption = "Magnitude"
m.add_child(colormap)
...

Ellipsis

In [11]:
quaqes = folium.map.FeatureGroup()
for index, row in NU_data.iterrows():
    quaqes.add_child(
        folium.features.CircleMarker(
            location=[row['LAT'],row['LON']],
            radius=5,
            color=colormap(row['MAG']),
            stroke=False,
            fill=True,
            fill_opacity=0.8,
            opacity=0,
            popup="year:{}\nmag:{}\nmagType:{}\ndepth:{}".format(row['DATE'].split('-')[0], row['MAG'], row['MAGTYPE'], row['DEPTH']),
        )
    )

m.add_child(quaqes)
...

Ellipsis

In [12]:
from branca.element import Figure
fig = Figure(width=800, height=500)
fig.add_child(m)